In [2]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, resample
import matplotlib.pyplot as plt

In [3]:
# Read the CSV file into the DataFrame
df = pd.read_csv("D:\shared_git\MaestriaThesis\FeaturesTabs\pp01_t1.csv")

# Rename the first four columns
new_column_names = ['channels', 'ID', 'Type', 'Epoch']
df.rename(columns=dict(zip(df.columns[:4], new_column_names)), inplace=True)

filtered_class_labels = df[df['channels'] == 5]
filtered_class_labels = filtered_class_labels.drop(columns=['ID'])
filtered_class_labels = filtered_class_labels.drop(columns=['channels', 'Epoch'])
filtered_class_labels = filtered_class_labels.reset_index(drop=True)

# Extract the next 1000 columns as an array
data_array = filtered_class_labels.iloc[:, 4:].values
class_labels = filtered_class_labels.iloc[:,0]

print(class_labels)

0        1
1        1
2        1
3        1
4        1
        ..
7542    12
7543    12
7544    12
7545    12
7546    12
Name: Type, Length: 7547, dtype: int64


In [4]:
mods = class_labels.copy() 
mod = ['Air', 'Vib', 'Car']
mods[(4 >= class_labels)] = mod[0]
mods[(5 <= class_labels) & (8 >= class_labels)] = mod[1]
mods[(9 <= class_labels) & (12 <= class_labels)] = mod[2]

print(mods.head(7000)) 
class_labels = (class_labels - 1) % 4 + 1 

0       Air
1       Air
2       Air
3       Air
4       Air
       ... 
6995    Car
6996    Car
6997    Car
6998    Car
6999    Car
Name: Type, Length: 7000, dtype: object


In [5]:
lowcut = 2.0  
highcut = 30.0  
fs_original = 250.0  
fs_downsampled = 60 

nyquist = 0.5 * fs_original
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(4, [low, high], btype='band')

filtered_data_array = np.apply_along_axis(lambda x: filtfilt(b, a, x), axis=1, arr=data_array)

downsample_factor = int(fs_original / fs_downsampled)
downsampled_data_array = resample(filtered_data_array, len(filtered_data_array[0]) // downsample_factor, axis=1)

In [6]:
from scipy.signal import butter, filtfilt, resample

row_index = 0
original_row = data_array[row_index, :]
filtered_row = filtered_data_array[row_index, :]
downsampled_row = downsampled_data_array[row_index, :]

window_size = 10
moving_average_row = np.convolve(downsampled_row, np.ones(window_size)/window_size, mode='same')

time_original = np.arange(0, len(original_row)) / fs_original

time_downsampled = np.arange(0, len(downsampled_row)) / fs_downsampled

time_moving_average = np.arange(0, len(moving_average_row)) / fs_downsampled

In [7]:
# Create a DataFrame with the downsampled data and class labels
combined_df = pd.concat([mods, pd.DataFrame(downsampled_data_array)], axis=1)

print(combined_df)

     Type         0         1         2         3         4         5  \
0     Air  0.551682  4.319338  8.119412  0.550700  1.641716  3.776759   
1     Air  2.295502  5.531810  6.669759  3.779851  6.212644  0.644145   
2     Air  0.951695  4.005640  5.720737  6.347818  2.466792  1.063216   
3     Air -0.633069 -4.673608 -0.309353 -1.826944  0.477519 -0.222302   
4     Air -0.811154 -4.235896 -2.731214  2.812897 -5.968457 -4.509219   
...   ...       ...       ...       ...       ...       ...       ...   
7542  Car -0.934849 -1.859289 -1.191566  2.239912  2.051117 -0.625381   
7543  Car  1.159886  3.406044  2.383993 -1.740570 -6.798391 -2.610167   
7544  Car  2.498982  9.069856  6.523309  8.431469  0.595698 -2.752342   
7545  Car  0.358859  2.839038  4.112990 -0.438341  2.349776  1.917421   
7546  Car  0.154401  2.665266  2.019970  1.930772  3.566385 -0.502490   

             6         7         8  ...       301       302       303  \
0     1.811655  0.371775  0.664142  ...  4.069009 

In [13]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Select features (X) and target variable (y)
X = downsampled_data_array  # Assuming the downsampled data starts from the 5th column
y = class_labels.values  # Assuming 'channels' is the target variable

y = y.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn_model = KNeighborsClassifier(n_neighbors=12)

knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 27.09%
